In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(r"El_Paso_NOAA_data\NOAA_El_Paso_1998-2000.csv")

# Need to have separate dataframes for every distinct year of NOAA data and write each year to csv

In [ ]:
set_of_stations = set()

for row in df.iterrows():
#     print(row[1]['STATION'])
    temp_tuple = (row[1]['STATION'], row[1]['LATITUDE'], row[1]['LONGITUDE'])
    set_of_stations.add(temp_tuple)

In [ ]:
len(set_of_stations)

In [ ]:
df = pd.read_excel(r"El_Paso_NOAA_data\NOAA_El_Paso_1998-2019.xlsx")

In [ ]:
df1 = df['DATE'].tolist()
len(df1)

In [ ]:
count = 0

for element in df1:
    if element.year == 2001:
        count += 1
        
print(count)

In [ ]:
count = 0

for year in range(1998, 2020):
    
    temp_list = []
    temp_set = set()
    
    for i, row in df.iterrows():
        print(i)
        
        if year == row['DATE'].year:
            temp_tuple = (row['STATION'], row['LATITUDE'], row['LONGITUDE'])
            temp_set.add(temp_tuple)
    
    my_list = list(temp_set)

    with open('weather_stations_{}.txt'.format(year), 'w') as fp:
        fp.write('\n'.join('{},{},{}'.format(x[0],x[1], x[2]) for x in my_list))

In [ ]:
len(temp_set)

In [ ]:
my_list = list(temp_set)

with open('test.txt', 'w') as fp:
    fp.write('\n'.join('{},{},{}'.format(x[0],x[1], x[2]) for x in my_list))

In [ ]:
df_merged = pd.concat(temp_list)

In [ ]:
df_merged.to_csv("test.csv")

In [ ]:
tempDataList = []

with open('El_Paso_Weather_Stations/weather_stations_2019.txt'.format(year), 'r') as f:
    for line in f:
#         print(line.split(','))
        tempDataList.append({'ID': line.split(',')[0], 'lat': line.split(',')[1], 'lon': line.split(',')[2].replace('\n', '')})

# Based on the Data_Exploration_Integration files, remove all rows that are not bridges located in El Paso County

# Using geolocator to search every coordinate pair to compare the county to see if it belongs to El Paso County or not would take approx. 15 - 18 hours so I approximated the bounding box of El Paso County to compare the numerical coordinates rather than use a geolocator which takes time

# I used: https://boundingbox.klokantech.com/ to form an approximate bounding box for El Paso County which returned min latitude: 254.929456, min longitude: 38.519019, max latitude: 255.948379, max longitude: 39.130161

# First run is for Data_Exploration_Integration_1.csv

In [19]:
def dms2dd(s):

    minutes = s[-6:-4]
    seconds = s[-4:]
    seconds_decimal = s[-4:-2] + "." +s[-2:]

    temp_dms = minutes
    temp_dms = temp_dms + seconds

    if temp_dms in s:
        degrees = s.replace(temp_dms,'')

    dd = float(degrees) + float(minutes)/60 + float(seconds_decimal)/(60*60);

    return dd

In [20]:
import pandas as pd
import os

from IPython.display import clear_output

file_num = 0

for idx in range(1, 10):
    
    file_num += 1
    
    NBI_df = pd.read_csv('Data_Exploration_Integration_{}.csv'.format(idx))

    NBI_df = NBI_df.dropna(subset=['LAT_016'])
    NBI_df = NBI_df.dropna(subset=['LONG_017'])

    NBI_df['LAT_016'] = NBI_df['LAT_016'].astype(int)
    NBI_df['LONG_017'] = NBI_df['LONG_017'].astype(int)

    decimal_lats = []
    decimal_longs = []

    for index, row in NBI_df.iterrows():
        
        str_lat = str(row["LAT_016"])
        str_long = str(row["LONG_017"])

        if len(str_lat) < 8 or len(str_long) < 8:
            NBI_df = NBI_df.drop([index])
            continue

        decimal_lat = dms2dd(str_lat)
        decimal_long = dms2dd(str_long)

        decimal_lats.append(decimal_lat)
        decimal_longs.append(decimal_long)

    decimal_longs = [360 - x for x in decimal_longs]

    NBI_df['LAT_016'] = decimal_lats
    NBI_df['LONG_017'] = decimal_longs

    count = 0

    for index, row in NBI_df.iterrows():
        count += 1
        print("file: " + str(file_num) + "iteration: " + str(count))
        clear_output(True)

        try:
            #min longitude: 254.929456, min latitude: 38.519019, max longitude: 255.948379, max latitude: 39.130161
            if 38.519019 > row['LAT_016'] or 39.130161 < row['LAT_016'] or 254.929456 > row['LONG_017'] or 255.948379 < row['LONG_017']:
                NBI_df = NBI_df.drop([index])
        except:
            NBI_df = NBI_df.drop([index])
            continue

    NBI_df.to_csv('Data_Exploration_Integration_El_Paso_{}.csv'.format(idx))

file: 1iteration: 13549


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [23]:
NBI_df

,Unnamed: 0,STATE_CODE_001,STRUCTURE_NUMBER_008,EVALUATION_YEAR,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,...,SUFFICIENCY_ASTERC,SUFFICIENCY_RATING,STATUS_NO_10YR_RULE,CAT10,CAT23,CAT29,SUBMITTED_BY,BRIDGE_CONDITION,LOWEST_RATING,DECK_AREA
7859,2533,8,DOU404-06.80-A,1998,1,4.0,1.0,0,0.0,15.0,...,NaN,99.9,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7860,2562,8,DOU404-06.80-A,1999,1,4.0,1.0,0.0,0.0,15.0,...,NaN,99.3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7861,2571,8,DOU404-06.80-A,2000,1,4.0,1.0,0.0,0.0,15.0,...,NaN,99.3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7862,2667,8,DOU404-06.80-A,2002,1,4.0,1.0,0,0.0,15.0,...,NaN,99.3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7865,2710,8,DOU404-06.80-A,2005,1,4.0,1.0,0,0.0,15.0,...,NaN,97.4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7866,2727,8,DOU404-06.80-A,2006,1,4.0,1.0,0,0.0,15.0,...,NaN,97.4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7867,2747,8,DOU404-06.80-A,2007,1,4.0,1.0,0,0.0,15.0,...,NaN,97.4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7868,2743,8,DOU404-06.80-A,2008,1,4.0,1.0,0,0.0,15.0,...,NaN,97.4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7869,2766,8,DOU404-06.80-A,2009,1,4.0,1.0,0,0.0,15.0,...,NaN,97.6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7870,2726,8,DOU404-06.80-A,2010,1,4.0,1.0,0,0.0,15.0,...,NaN,97.6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
from geopy.geocoders import Nominatim

locator = Nominatim(user_agent="myGeocoder")
location = locator.reverse((NBI_df['LAT_016'][7913], NBI_df['LONG_017'][7913]))
location.raw['address']['county']

'El Paso County'